<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-5/blob/main/ML_Zoomcamp_Week_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 03/10/2022**